# Predict

In [20]:
AWS_ACCESS_KEY_ID="minio123"
AWS_SECRET_ACCESS_KEY="minio123"
MLFLOW_S3_ENDPOINT_URL="http://localhost:9990"

from os import environ

environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL

In [19]:
import mlflow
import mlflow.tensorflow
import tensorflow as tf

mlflow.set_tracking_uri("http://localhost:5000")

# Replace '<run_id>' with the actual run ID where your model is stored
model_uri = "runs:/1c81e50024c54dd69d8e48bc406c4dad/model"

# Load the TensorFlow model from MLflow
model = mlflow.tensorflow.load_model(model_uri)

from qutip import basis, tensor, rand_ket
import numpy as np
from simulation_utils import flatten_density_matrix

entangled = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))) / np.sqrt(2)
not_entangled = tensor(rand_ket(2), rand_ket(2)).unit()

input_data = flatten_density_matrix(entangled * entangled.dag())
predictions = model.predict(tf.constant([input_data]))
print(predictions)

input_data = flatten_density_matrix(not_entangled * not_entangled.dag())
predictions = model.predict(tf.constant([input_data]))
print(predictions)

2024/06/02 14:57:08 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

I0000 00:00:1717333028.330239  176050 service.cc:145] XLA service 0x7ff69c002fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1717333028.330259  176050 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2024-06-02 14:57:08.333310: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-02 14:57:08.343320: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1717333028.380476  176050 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[[5.702452e-11]]
